In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(1)
sns.set_palette('flare')

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [2]:
features, target = load_wine(return_X_y = True, as_frame = True)
df = pd.concat([features, target], axis = 1)
df.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0


In [3]:
classes = df['target'].unique()
classes

array([0, 1, 2])

In [4]:
def normalize(data):
    for column in data.columns[:-1]:
        data[column] = (data[column] - data[column].mean()) / data[column].std()

In [5]:
normalize(df)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,1.514341,-0.560668,0.231400,-1.166303,1.908522,0.806722,1.031908,-0.657708,1.221438,0.251009,0.361158,1.842721,1.010159,0
1,0.245597,-0.498009,-0.825667,-2.483841,0.018094,0.567048,0.731565,-0.818411,-0.543189,-0.292496,0.404908,1.110317,0.962526,0
2,0.196325,0.021172,1.106214,-0.267982,0.088110,0.806722,1.212114,-0.497005,2.129959,0.268263,0.317409,0.786369,1.391224,0
3,1.686791,-0.345835,0.486554,-0.806975,0.928300,2.484437,1.462399,-0.979113,1.029251,1.182732,-0.426341,1.180741,2.328007,0
4,0.294868,0.227053,1.835226,0.450674,1.278379,0.806722,0.661485,0.226158,0.400275,-0.318377,0.361158,0.448336,-0.037767,0


In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = pd.DataFrame(ohe.fit_transform(df[['target']]).toarray())
df_ohe = df.join(y)
df_ohe.drop(['target'], axis = 1, inplace = True)
df_ohe.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,0,1,2
0,1.514341,-0.560668,0.231400,-1.166303,1.908522,0.806722,1.031908,-0.657708,1.221438,0.251009,0.361158,1.842721,1.010159,1.0,0.0,0.0
1,0.245597,-0.498009,-0.825667,-2.483841,0.018094,0.567048,0.731565,-0.818411,-0.543189,-0.292496,0.404908,1.110317,0.962526,1.0,0.0,0.0
2,0.196325,0.021172,1.106214,-0.267982,0.088110,0.806722,1.212114,-0.497005,2.129959,0.268263,0.317409,0.786369,1.391224,1.0,0.0,0.0
3,1.686791,-0.345835,0.486554,-0.806975,0.928300,2.484437,1.462399,-0.979113,1.029251,1.182732,-0.426341,1.180741,2.328007,1.0,0.0,0.0
4,0.294868,0.227053,1.835226,0.450674,1.278379,0.806722,0.661485,0.226158,0.400275,-0.318377,0.361158,0.448336,-0.037767,1.0,0.0,0.0


In [7]:
X = np.array(df_ohe.iloc[:, :-3], dtype = 'float')
y = np.array(y, dtype = 'float')
m, n = X.shape

b = np.ones((m, 1), dtype = 'float')
X = np.column_stack((b, X))

print(X.shape, y.shape)

(178, 14) (178, 3)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(142, 14) (36, 14) (142, 3) (36, 3)


In [9]:
def h(X, theta):
    return 1 / (1 + np.exp(np.dot(X, theta)))

In [10]:
def J(X, y, theta):
    return -(1/m) * (np.dot(y.T, np.log(h(X, theta))) + np.dot((1 - y).T, np.log(1 - h(X, theta))))

In [54]:
def gradient_descent(X, y, theta, learning_rate, epochs, batch_size):
    
    cost = []
    it = []
    i = 1
    thetas = []
    cost_class1 = []
    cost_class2 = []
    cost_class3 = []
    
    for i in range(1, epochs + 1):
        
        cost_class1.append(J(X, y, theta))
        it.append(i)
        
        dt = (1/m) * np.dot(X.T, h(X, theta) - y)
        theta = theta - learning_rate * dt
        thetas.append(theta)
       
    print('Final Cost:', J(X, y, theta))
#     weight_plot(thetas, it, title)
#     cost_plot(cost, it, title)
    print('Final Weights: ', theta)
    return theta

In [55]:
theta = np.random.randn(n + 1, 1)
print((h(X, theta) - y).shape)
print(np.dot(X.T, h(X, theta) - y).shape)

(178, 3)
(14, 3)


In [11]:
theta = np.random.randn(n + 1, 1)

In [12]:
def cost_plot(cost, iterations, title):
    
    plt.xlabel('Iterations')
    plt.ylabel('Cost')
    plt.plot(iterations, cost)
    plt.title(title)
    
    plt.show()

In [18]:
# for i in range(0, y.shape[1]):
#     theta = gradient_descent(X_train, y_train[])
print(y_train)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1.